<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Big_Food_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pips install
#!pip install evidently
#!pip install facets-overview
#!pip install yfinance

#!pip install itables # for interactive dataframe display

In [12]:
import  pandas as pd

#libaries for display and interactivity
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

from IPython.core.display import display, HTML
from google.colab import output
output.enable_custom_widget_managet()

from ipywidgets import interact, interactive
import ipywidgets as widgets

from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab


#Open Data-set on Food-Nutrition-health from research sector

***FRIDA Food Composition Database: ***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [10]:
# Import FRIDA NNutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head(3)

,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration","Kulhydrat, difference",...,FCF,Alkohol,Aske,Tørstof,Vand,A-vitamin,Retinol,beta-caroten,D-vitamin,D3-vitamin
Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,g,...,-,g,g,g,g,RE,µg,µg,µg,µg
1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0


In [48]:
row=data_frida[data_frida.Navn== "Kylling, bryst (filet), kogt, pålæg"]
print("Id: ", row.index[0])
print("Category: ", row.Gruppe.values[0])
print("Name: ", row.Navn.values[0])
print("Total Energy: ", row.Energy_kj.values[0], "Kj (", row.Energy_kcal.values[0], "Kacl)")
print("Total Energy: ", row.Energy_kcal.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])

Id:  561
Category:  Kylling og høne
Name:  Kylling, bryst (filet), kogt, pålæg
Total Energy:  475 Kj ( 113 Kacl)
Total Energy:  113
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475


In [11]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)

# Assign columns name that defined just above
data_frida.columns= column_names

# lets see somerowsof the data
data_frida.head(3)

,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,Kulhydrat_difference_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0


**FlavourDB: Database of flavor molecules**
*   link ="https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/methods-and-application-of-food-composition-laboratory/mafcl-site-pages/flavonoids/"
*   Academic Article= https://academic.oup.com/nar/article/46/D1/D1210/4559748
*   *FlavourDB= https://cosylab.iiitd.edu.in/flavordb/*

***Food Database called "FoodDB"***

for inspiiration
Link= https://foodb.ca/foods/FOOD00008


In [104]:
from google.colab import files
uploaded = files.upload()

Saving OntologyTerm.json to OntologyTerm.json


In [67]:
type(uploaded)

dict

In [106]:
#Food Database called "FoodDB"

import os
import json
import pandas as pd

# # Get the list of all files and directories with JSON files
#food_db_dir= "/content/FoodDB_JSON.zip" #"C:/Users/Bruger/Desktop/dataset/FoodDB_JSON/" # The directory with the JSON files
link= "/content/OntologyTerm.json"
data = pd.read_json(link, lines=True)
data.head(3)

,id,term,definition,external_id,external_source,comment,curator,parent_id,level,created_at,updated_at,legacy_id
0,7659,Process,"Biological or chemical events, or a series the...",None,None,None,None,NaN,1.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN
1,7660,Naturally occurring process,Naturally-occurring molecular events or a seri...,None,None,None,None,7659.0,2.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN
2,7661,Biological process,Biological or chemical events or a series ther...,None,None,None,None,7660.0,3.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN


***FoodOn: A farm to fork ontology***

*   link= https://foodon.org/

> Application of FoodOn: https://foodon.org/wp-content/uploads/2020/11/image-768x310.png 

*   FoodKG: A Semantics-Driven Knowledge Graph for Food Recommendation:https://foodkg.github.io/subs.html

In [ ]:
# FoodOn:A harmonized food ontology to increase global food traceability


***FoodOntomap:* Linking food concepts across Different Food Ontologies**
Link of Database: https://zenodo.org/record/3600619/files/FoodOntoMap_v2.zip?download=1


***Food Image datasets:***

*   **link:** https://www.epfl.ch/labs/mmspg/downloads/food-image-datasets/
*   **Food-5K:** Dataset of food-imgesand non-food images, resulting in a totoal of 5,000 images
*   **Food-11:** dataset consist of 16,643 images grouped into 11 categories, which basiically covers the major types of food that people consumer in daily life

*** FAO Food Composition Databases ***
*   Link: https://www.fao.org/infoods/infoods/tables-and-databases/faoinfoods-databases/en/
*   List item



***NutriChem 2.0:* exploring the effect of plant-based foods on human health and drug efficacy**

***DietHub:* Dietary habits analysis through understanding the content of recipes**

***US food Data central ***
API key= https://api.nal.usda.gov/fdc/v1/foods/search?query=apple&pageSize=2&api_key=JdC6qVfCHr3AEEaiKkQrUFZGrXncNUGup7E5crN1